In [19]:
import pandas as pd
import numpy as np
# import random as rn
# import sklearn
# from scipy import stats
# import math
import re

from sklearn.preprocessing import  LabelEncoder #OneHotEncoder
# from sklearn import ensemble
from sklearn import linear_model

# from sklearn.model_selection import GridSearchCV,cross_val_score
# from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error,accuracy_score

# # from sklearn.ensemble import RandomForestClassifier
# from sklearn import linear_model

# import matplotlib.pyplot as plt
# import seaborn as sns
# from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, RidgeCV, LassoCV

import pickle

In [2]:
totDF = pd.read_csv('../../data/processed/Cleaned_Data_Set.csv')

In [18]:
totDF.groupby(['birth_year','admit_NICU']).count()

birth_month  birth_time  birth_day_of_wk  birth_place  \
birth_year admit_NICU                                                          
2014       N                176434      176434           176434       176434   
           U                  7688        7688             7688         7688   
           Y                 15878       15878            15878        15878   
2015       N                179494      179494           179494       179494   
           U                  3805        3805             3805         3805   
           Y                 16701       16701            16701        16701   
2016       N                182334      182334           182334       182334   
           U                   228         228              228          228   
           Y                 17438       17438            17438        17438   
2017       N                181688      181688           181688       181688   
           U                   178         178              178          178   
           Y                 18134       18134            18134        18134   
2018       N                181525      181525           181525       181525   
           U                   185         185              185          185   
           Y                 18290       18290            18290        18290   

                       mothers_age  mothers_nativity  \
birth_year admit_NICU                                  
2014       N                176434            176434   
           U                  7688              7688   
           Y                 15878             15878   
2015       N                179494            179494   
           U                  3805              3805   
           Y                 16701             16701   
2016       N                182334            182334   
           U                   228               228   
           Y                 17438             17438   
2017       N                181688            181688   
           U                   178               178   
           Y                 18134             18134   
2018       N                181525            181525   
           U                   185               185   
           Y                 18290             18290   

                       mothers_residence_status  mothers_race  \
birth_year admit_NICU                                           
2014       N                             176434        176434   
           U                               7688          7688   
           Y                              15878         15878   
2015       N                             179494        179494   
           U                               3805          3805   
           Y                              16701         16701   
2016       N                             182334        182334   
           U                                228           228   
           Y                              17438         17438   
2017       N                             181688        181688   
           U                                178           178   
           Y                              18134         18134   
2018       N                             181525        181525   
           U                                185           185   
           Y                              18290         18290   

                       mothers_hispanic_origin  mothers_hispanic_origin2  ...  \
birth_year admit_NICU                                                     ...   
2014       N                            176434                    176434  ...   
           U                              7688                      7688  ...   
           Y                             15878                     15878  ...   
2015       N                            179494                    179494  ...   
           U                              3805                      3805  ...   
           Y                        

## Cleaning / Sampling

In [7]:
def cleanDF (df):
    r1 = re.compile('.*reporting')
    r2 = re.compile('.*imputed')

    cols_to_drop1 = list(filter((r1.match), df.columns))
    cols_to_drop2 = list(filter((r2.match), df.columns))
    cols_to_drop3 = ['admit_NICU']
    cols_to_drop = cols_to_drop1 + cols_to_drop2 + cols_to_drop3

    cols_to_keep = [col for col in df.columns if col not in cols_to_drop]

    X_and_target = df[cols_to_keep + ['admit_NICU']].copy()

    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    catDF = X_and_target.select_dtypes(include=object).copy()
    numDF = X_and_target.select_dtypes(include=numerics).copy() #only numeric columns

    le = LabelEncoder()
    catDF = catDF.apply(le.fit_transform)

    concat_df = pd.concat([numDF,catDF],axis=1)
    return concat_df

## Logistic Model Part 1

In [4]:
sample_size_list = [100]

In [5]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#GLM with Cross Validation
for sample_per_year in sample_size_list:
    dwnSmplDF = concat_df.groupby('birth_year',group_keys = False).apply(lambda x: x.sample(sample_per_year))
    
    cl_df = dwnSmplDF[cols_to_keep]
    encoded_target = dwnSmplDF['admit_NICU']
    
    glm_CV = linear_model.LogisticRegressionCV(#Cs = int(1e4),
                                                cv = 5,
                                                solver = 'saga',
                                                n_jobs = -1,
                                                random_state = 108
                                               ).fit(cl_df, encoded_target)
    print('sample size : %d\n' % (sample_per_year*5))
    %time glm_CV.fit(cl_df, encoded_target)
    print('\nscore    : {0}'.format(glm_CV.score(cl_df, encoded_target)))
    print('-'*50)


In [ ]:
'''
sample size : 500

CPU times: user 6min 47s, sys: 1min 26s, total: 8min 14s
Wall time: 1min 54s

score    : 0.932
--------------------------------------------------
'''

In [ ]:
#GLM with Lasso Penalty

for sample_per_year in sample_size_list:
    dwnSmplDF = concat_df.groupby('birth_year',group_keys = False).apply(lambda x: x.sample(sample_per_year))
    
    cl_df = dwnSmplDF[cols_to_keep]
    encoded_target = dwnSmplDF['admit_NICU']
    
    glm_lasso = linear_model.LogisticRegression(penalty = 'l1', 
                                              solver = 'saga', 
                                              multi_class='auto', 
                                              n_jobs = -1,
                                              C = 1e4)
    print('sample size : %d\n' % (sample_per_year*5))
    %time glm_lasso.fit(cl_df, encoded_target)
    print('\nscore    : {0}'.format(glm_lasso.score(cl_df, encoded_target)))
    print('-'*50)
    

In [7]:
#GLM with Lasso Penalty and Cross Validation

sample_size_list = [100,1000,10000]

for sample_per_year in sample_size_list:
    dwnSmplDF = concat_df.groupby('birth_year',group_keys = False).apply(lambda x: x.sample(sample_per_year))
    cl_df = dwnSmplDF[cols_to_keep]
    encoded_target = dwnSmplDF['admit_NICU']
    
    glm_lassoCV = linear_model.LogisticRegressionCV(#Cs = int(1e4),
                                                cv = 5,
                                                penalty = 'l1',
                                                solver = 'saga',
                                                n_jobs = -1,
                                                random_state = 108
                                               ).fit(cl_df, encoded_target)
    print('sample size : %d\n' % (sample_per_year*5))
    %time glm_lassoCV.fit(cl_df, encoded_target)
    print('\nscore    : {0}'.format(glm_lassoCV.score(cl_df, encoded_target)))
    print('-'*50)
    

sample size : 500

CPU times: user 2.87 s, sys: 97.3 ms, total: 2.96 s
Wall time: 571 ms

score    : 0.914
--------------------------------------------------
sample size : 5000

CPU times: user 2min, sys: 577 ms, total: 2min 1s
Wall time: 17.4 s

score    : 0.9308
--------------------------------------------------
sample size : 50000

CPU times: user 22min 12s, sys: 2.49 s, total: 22min 14s
Wall time: 3min 17s

score    : 0.9245
--------------------------------------------------


In [26]:
#GLM with Lasso Penalty and Cross Validation

sample_size_list = [20000]

for sample_per_year in sample_size_list:
    dwnSmplDF = concat_df.groupby('birth_year',group_keys = False).apply(lambda x: x.sample(sample_per_year))
    cl_df = dwnSmplDF[cols_to_keep]
    encoded_target = dwnSmplDF['admit_NICU']
    
    glm_lassoCV = linear_model.LogisticRegressionCV(#Cs = int(1e4),
                                                cv = 5,
                                                penalty = 'l1',
                                                solver = 'saga',
                                                n_jobs = -1,
                                                random_state = 108
                                               ).fit(cl_df, encoded_target)
    print('sample size : %d\n' % (sample_per_year*5))
    %time glm_lassoCV.fit(cl_df, encoded_target)
    print('\nscore    : {0}'.format(glm_lassoCV.score(cl_df, encoded_target)))
    print('-'*50)

sample size : 100000

CPU times: user 50min 24s, sys: 20.1 s, total: 50min 45s
Wall time: 8min 27s

score    : 0.92502
--------------------------------------------------


In [27]:
glm_lassoCV

LogisticRegressionCV(Cs=10, class_weight=None, cv=5, dual=False,
                     fit_intercept=True, intercept_scaling=1.0, l1_ratios=None,
                     max_iter=100, multi_class='warn', n_jobs=-1, penalty='l1',
                     random_state=108, refit=True, scoring=None, solver='saga',
                     tol=0.0001, verbose=0)

In [28]:
pickle.dump(glm_lassoCV, open('best_glmlassoCV.sav', 'wb'))
# glm_lassoCV = pickle.load(open('best_glmlassoCV.sav', 'rb'))

In [10]:
glm_lassoCV.get_params

<bound method BaseEstimator.get_params of LogisticRegressionCV(Cs=10, class_weight=None, cv=5, dual=False,
                     fit_intercept=True, intercept_scaling=1.0, l1_ratios=None,
                     max_iter=100, multi_class='warn', n_jobs=-1, penalty='l1',
                     random_state=108, refit=True, scoring=None, solver='saga',
                     tol=0.0001, verbose=0)>

In [30]:
from sklearn.metrics import confusion_matrix

print(confusion_matrix(encoded_target,y_pred))

[[89917     5   355]
 [  151  1083     6]
 [ 6981     0  1502]]


In [31]:
cf = confusion_matrix(encoded_target,y_pred)

In [38]:
np.set_printoptions(suppress=True)
print(cf/1000.)

[[89.917  0.005  0.355]
 [ 0.151  1.083  0.006]
 [ 6.981  0.     1.502]]


In [43]:
glmLCV_coefs = pd.DataFrame({'col' :list(cl_df.columns), 
                           'coef0': glm_lassoCV.coef_[0], 
                          'coef1': glm_lassoCV.coef_[1],
                          'coef2': glm_lassoCV.coef_[2]})

In [44]:
glmLCV_coefs

,col,coef0,coef1,coef2
0,birth_year,-0.002271,-0.004909,0.002863
1,birth_month,-0.000024,0.000033,0.000066
2,birth_time,-0.000175,0.000630,-0.000035
3,birth_day_of_wk,-0.000594,-0.000011,0.000359
4,birth_place,-0.001830,0.000313,-0.001112
...,...,...,...,...
98,infant_living_at_report,-0.000705,-0.000051,0.001135
99,infant_breastfed_at_discharge,0.002098,0.000009,-0.002149
100,cigs_before,0.000508,0.005762,-0.001221
101,mothers_bmi_recode,-0.000593,0.000186,0.000755


In [45]:
glmLCV_coefs['abs_coef0'] = glmLCV_coefs['coef0'].apply(abs)
glmLCV_coefs['abs_coef1'] = glmLCV_coefs['coef1'].apply(abs)
glmLCV_coefs['abs_coef2'] = glmLCV_coefs['coef2'].apply(abs)

In [46]:
top20_coef0 = glmLCV_coefs.nlargest(10,'abs_coef0')['col']
top20_coef1 = glmLCV_coefs.nlargest(10,'abs_coef1')['col']
top20_coef2 = glmLCV_coefs.nlargest(10,'abs_coef2')['col']

In [47]:
top20_coef0

77           obst_est_edit_wk
76      combined_gestation_wk
43               n_prev_cesar
60      final_delivery_method
71          APGAR_score_10min
79          assist_vent_immed
82    antibiotics_for_newborn
5                 mothers_age
8                mothers_race
56                antibiotics
Name: col, dtype: object

In [48]:
top20_coef1

43                  n_prev_cesar
25             n_prenatal_visits
24        mo_prenatal_care_began
100                  cigs_before
0                     birth_year
102                  weight_gain
8                   mothers_race
96     no_cong_anamolies_checked
29                     cigs_tri3
28                     cigs_tri2
Name: col, dtype: object

In [49]:
top20_coef2

77           obst_est_edit_wk
76      combined_gestation_wk
71          APGAR_score_10min
60      final_delivery_method
79          assist_vent_immed
82    antibiotics_for_newborn
5                 mothers_age
43               n_prev_cesar
8                mothers_race
56                antibiotics
Name: col, dtype: object

In [50]:
list(set(top20_coef0).union(set(top20_coef1)).union(set(top20_coef2)))

['assist_vent_immed',
 'antibiotics_for_newborn',
 'birth_year',
 'assist_vent_after6',
 'mo_prenatal_care_began',
 'prior_terminations',
 'mothers_age',
 'final_delivery_method',
 'n_prev_cesar',
 'weight_gain',
 'obst_est_edit_wk',
 'cigs_tri2',
 'mothers_race',
 'cigs_before',
 'cigs_tri3',
 'combined_gestation_wk',
 'mothers_bmi',
 'n_prenatal_visits',
 'APGAR_score_10min']

In [51]:
list(set(top20_coef0).intersection(set(top20_coef1)).intersection(set(top20_coef2)))

['n_prev_cesar', 'mothers_race']

In [ ]:
'''
sample size : 100000

CPU times: user 54min 1s, sys: 37.3 s, total: 54min 39s
Wall time: 8min 39s

score    : 0.92512
--------------------------------------------------
'''

In [ ]:
#GLM with Lasso Penalty and Cross Validation

# sample_size_list = [200000]

# for sample_per_year in sample_size_list:
#     dwnSmplDF = concat_df.groupby('birth_year',group_keys = False).apply(lambda x: x.sample(sample_per_year))
#     cl_df = dwnSmplDF[cols_to_keep]
#     encoded_target = dwnSmplDF['admit_NICU']
    
#     glm_lassoCV = linear_model.LogisticRegressionCV(#Cs = int(1e4),
#                                                 cv = 5,
#                                                 penalty = 'l1',
#                                                 solver = 'saga',
#                                                 n_jobs = -2,
#                                                 random_state = 108
#                                                ).fit(cl_df, encoded_target)
#     print('sample size : %d\n' % (sample_per_year*5))
#     %time glm_lassoCV.fit(cl_df, encoded_target)
#     print('\nscore    : {0}'.format(glm_lassoCV.score(cl_df, encoded_target)))
#     print('-'*50)
    

## Logistic Model Part 2

Sampled in a way that
1. Unknowns in `admit_NICU` column was thrown away.
2. There are equal number of `Y`'s and `N`'s in `admit_NICU` column. (balanced sampling)

In [10]:
cl_df = cleanDF(totDF)
nicu_allY = cl_df.loc[cl_df['admit_NICU']==1]
nicu_allN = cl_df.loc[cl_df['admit_NICU']==0]

In [9]:
#pure GLM with balanced sample (w/o stratified year)
sample_size_list = [100]

for sample_per_class in sample_size_list:

    sampN = nicu_allN.sample(sample_per_class)
    sampY = nicu_allY.sample(sample_per_class)
    samp = pd.concat([sampN,sampY],axis=0)
    
    samp_target = samp.admit_NICU
    samp_X = samp.drop('admit_NICU',axis=1)
    
#     bal_dwnSmplY = nicu_allY.groupby('birth_year',group_keys = False).apply(lambda x: x.sample(sample_per_year))
#     bal_dwnSmplN = nicu_allN.groupby('birth_year',group_keys = False).apply(lambda x: x.sample(sample_per_year))
#     bal_dwnSmpl = pd.concat([bal_dwnSmlpY,bal_dwnamlpN],axis=0)
    
    glm = linear_model.LogisticRegression(solver = 'saga', 
                                              multi_class='auto', 
                                              n_jobs = -1,
                                              C = 1e4)
    print('sample size : %d\n' % (sample_per_class*2))
    %time glm.fit(samp_X, samp_target)
    print('\nscore    : {0}'.format(glm.score(samp_X, samp_target)))
    print('-'*50)
    

sample size : 200

CPU times: user 19.3 ms, sys: 6.91 ms, total: 26.2 ms
Wall time: 115 ms

score    : 0.925
--------------------------------------------------


In [17]:
#pure GLM with balanced sample (w/ stratified year)

sample_size_class = [100]

for sample_per_class in sample_size_class:

#     sampN = nicu_allN.sample(sample_per_class)
#     sampY = nicu_allY.sample(sample_per_class)
#     samp = pd.concat([sampN,sampY],axis=0)
    
#     samp_target = samp.admit_NICU
#     samp_X = samp.drop('admit_NICU',axis=1)
    
    bal_dwnSmplY = nicu_allY.groupby('birth_year',group_keys = False).apply(lambda x: x.sample(sample_per_class))
    bal_dwnSmplN = nicu_allN.groupby('birth_year',group_keys = False).apply(lambda x: x.sample(sample_per_class))
    bal_dwnSmpl = pd.concat([bal_dwnSmplY,bal_dwnSmplN],axis=0)
    
    bal_target = bal_dwnSmpl.admit_NICU
    bal_X = bal_dwnSmpl.drop('admit_NICU',axis=1)
    
    glm = linear_model.LogisticRegression(solver = 'saga', 
                                              multi_class='auto', 
                                              n_jobs = -1,
                                              C = 1e4)
    print('sample size : %d\n' % (sample_per_class*2))
    %time glm.fit(bal_X, bal_target)
    print('\nscore    : {0}'.format(glm.score(bal_X, bal_target)))
    print('-'*50)
    

sample size : 200

CPU times: user 79.9 ms, sys: 1.13 ms, total: 81 ms
Wall time: 108 ms

score    : 0.774
--------------------------------------------------
